In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from scipy import ndimage
import numpy as np
from tensorflow import keras
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D,Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
from keras.layers import BatchNormalization,ReLU,GlobalAveragePooling1D,MaxPooling1D,LSTM,TimeDistributed,GlobalAveragePooling2D
#from keras.utils import to_categorical
from tensorflow.keras.models import save_model,load_model
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model,save_model,load_model
from keras.layers import Input
from keras.layers.merge import concatenate

from datetime import datetime
from packaging import version

import os
import tempfile
import os
import tensorflow as tf
from tensorflow import expand_dims
from tensorflow import keras

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def load_file(filepath):
  dataframe=read_csv(filepath,header=None,delim_whitespace=True)
  return dataframe.values

In [3]:
def load_group(filenames,prefix=''):
  loaded=list()
  for name in filenames:
    data=load_file(prefix+name)
    loaded.append(data)
  loaded=dstack(loaded)
  return loaded

In [4]:
def load_dataset_group(group,prefix=''):
  filepath=prefix+group+'/Inertial Signals/'
  filenames=list()
#  filenames+=['total_acc_x_'+group+'.txt','total_acc_y_'+group+'.txt','total_acc_z_'+group+'.txt']
  filenames+=['body_acc_x_'+group+'.txt','body_acc_y_'+group+'.txt','body_acc_z_'+group+'.txt']
# filenames+=['body_gyro_x_'+group+'.txt','body_gyro_y_'+group+'.txt','body_gyro_z_'+group+'.txt']
  X=load_group(filenames,filepath)
  y=load_file(prefix+group+'/y_'+group+'.txt')
  return (X,y)

In [8]:
def load_dataset(prefix=''):
  trainX,trainy=load_dataset_group('train',prefix+'F:/HAR/UCI/')
  print(trainX.shape,trainy.shape)
  testX,testy=load_dataset_group('test',prefix+'F:/HAR/UCI/')
  print(testX.shape,testy.shape)
  trainy=trainy-1
  testy=testy-1
  #trainy=tf.compat.v1.keras.utils.to_categorical(trainy)
  #testy=tf.compat.v1.keras.utils.to_categorical(testy)
  print(trainX.shape, trainy.shape, testX.shape, testy.shape)
  return trainX, trainy, testX, testy

In [9]:
from sklearn.preprocessing import StandardScaler

def scale_data(trainX, testX):
	# remove overlap
	cut = int(trainX.shape[1] / 2)
	longX = trainX[:, -cut:, :]
	# flatten windows
	longX = longX.reshape((longX.shape[0] * longX.shape[1], longX.shape[2]))
	# flatten train and test
	flatTrainX = trainX.reshape((trainX.shape[0] * trainX.shape[1], trainX.shape[2]))
	flatTestX = testX.reshape((testX.shape[0] * testX.shape[1], testX.shape[2]))
	# standardize

	s = StandardScaler()
		# fit on training data
	s.fit(longX)
		# apply to training and test data
	longX = s.transform(longX)
	flatTrainX = s.transform(flatTrainX)
	flatTestX = s.transform(flatTestX)
	# reshape
	flatTrainX = flatTrainX.reshape((trainX.shape))
	flatTestX = flatTestX.reshape((testX.shape))
	return flatTrainX, flatTestX

In [10]:
trainX,trainy,testX,testy=load_dataset()

(7352, 128, 3) (7352, 1)
(2947, 128, 3) (2947, 1)
(7352, 128, 3) (7352, 1) (2947, 128, 3) (2947, 1)
